# Data201 Group Project

This project looks at the impact of covid-19 on New Zealand by looking at correlations between covid case numbers and various
other societal factors that have been impacted.

Installing libraries used throughout the project

In [1]:
library(tidyverse) 
library(ggplot2) # For plotting data
library(stringr)
library(visdat)
library(readxl)


library(rvest) # For webscraping
library(purrr) # For dataframe mapping
library(polite) # polite is the "polite" version of rvest
library(xml2) # makes it easier to work with HTML and XML from R

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5          ✔ purrr   0.3.4     
✔ tibble  3.1.5          ✔ dplyr   1.0.7.9000
✔ tidyr   1.1.3          ✔ stringr 1.4.0     
✔ readr   2.0.1          ✔ forcats 0.5.1     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding




# Downloading and Setting up data
This first section of the notebook downloads and does some basic wrangling of all of the data used throughout the project

# Getting the covid case data setup

The global covid case data we are using comes from the WHO, whose global covid data csv we have uploaded to our github.
This section simply retrieves the data from our github, and does some simple wrangling to extract the data that is relevant for our other datasets

In [2]:
# Reading data into tibble
case_data <- read_csv('https://raw.githubusercontent.com/andyholmes1210/DATA201-Project/main/WHO-COVID-19-global-data.csv')

# Selecting only the case data relevant to New Zealand, as well as only the date and new case count
case_data <- case_data %>%
    filter(Country == "New Zealand") %>%
    select(Date_reported, New_cases)

Rows: 155709 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): Country_code, Country, WHO_region
dbl  (4): New_cases, Cumulative_cases, New_deaths, Cumulative_deaths
date (1): Date_reported

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Getting the covid vaccination data setup

The covid vaccination data comes directly from the NZ ministry of health github.
This section downloads the nz ministry of health vaccination doses data per dhb group

In [3]:
vaccination_data_raw <- 'https://raw.githubusercontent.com/minhealthnz/nz-covid-data/main/vaccine-data/latest/doses_group_and_dhb_service.csv' %>% 
    read_csv()

Rows: 3653 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): Group, DHB of service
dbl  (2): Dose number, # doses administered
date (1): Week ending

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Getting the regional population data setup
This whole section scrapes the NZ regional DHB population data from the ministry of health website.
This combines the Auckland, Counties Manakau and Waitemata data into one region (Auckland), as well as the Capital & Coast and Hutt valley regions into one.
Source: https://www.health.govt.nz/new-zealand-health-system/my-dhb

In [4]:
# Main url that the population data is scraped from
base_url <- 'https://www.health.govt.nz'
main_location <- '/new-zealand-health-system/my-dhb'

main_page <- read_html(paste(base_url, main_location, sep=""))

# Getting the individual page elements
DHBs <- main_page %>%
    html_elements('#block-system-main > div > div > div.panel-panel.left.col-xs-12.col-md-3.col-lg-3 > div > div > div > ul') %>%
    html_children()


# Getting the names of the DHBS from each of the elements
DHB_names <- DHBs %>%
    html_text() %>%
    str_replace_all(' DHB', '')


# Getting the DHB specific page links
DHB_sub_pages <- DHBs %>%
    html_children() %>%
    html_attr('href') %>%
    paste(base_url, ., sep="") %>%
    str_trim() %>%
    str_squish()

# Combining the Region and Link info into a dataframe
population_info <- tibble(Region = DHB_names,
                          Link = DHB_sub_pages)

In [5]:
# This function takes a link to a specific DHB page and returns the population
get_population <- function(link) {
    
    # Reading the page and getting the bulk text object (I cant just select a specific text block as the position of the information varies from page to page)
    page <- read_html(link)
    text <- page %>%
        html_elements('.field-item.even') %>%
        .[[2]] %>%
        html_children() %>%
        html_text()
    
    # Joining the list of text boxes into one large string
    combined_text <- text %>%
        paste(collapse = '') %>%
        .[[1]] %>%
        .[[1]] 

   # The layout on the auckaland page differs slightly, so the retrieving of the population number differs for it.
    if(grepl('auckland', link, fixed=TRUE)) {
        population <- combined_text %>%
        str_split('It has a population of') %>%
        .[[1]] %>%
        .[[2]] %>%
        str_split(' ') %>%
        .[[1]] %>%
        .[[1]] %>%
        str_trim() %>%
        str_split(intToUtf8(160)) %>%
        .[[1]] %>%
        .[[1]]
    } else{
        population <- combined_text %>%
        str_split('It has a population of') %>%
        .[[1]] %>%
        .[[2]] %>%
        str_split(' ') %>%
        .[[1]] %>%
        .[[2]]
        
    }
    # Parsing the population as a number
    population <- population %>%
        str_replace_all(',', '') %>%
        str_replace_all(' ', '') %>%
        as.numeric()
    
    return(population)
}

In [6]:
# Adding the population data into the dataframe
population_df <- population_info %>% 
    mutate(Population = map_dbl(Link, get_population)) %>%
    mutate(Region = str_trim(Region)) %>%
    mutate(Region = str_trim(Region))


In [7]:
# This cell combines the Auckland, Counties Manakau and Waitemata into one large Auckland group, and combines Capital & Coast and Hutt Valley into one group.

# List of the Component DHBS
auckland_dhbs <- c('Auckland', 'Counties Manukau', 'Waitematā')
capital_coast_hutt_dhbs <- c('Capital & Coast', 'Hutt Valley')

# Getting the total population of the auckland DHBS
auckland <- population_df %>% 
    filter(Region %in% auckland_dhbs)
auckland_population <- sum(auckland$Population)

# Getting the total population of the capital and coast and hutt valley DHBS
capital_hutt <- population_df %>%
    filter(Region %in% capital_coast_hutt_dhbs)
capital_hutt_population <- sum(capital_hutt$Population)

# Replacing the component entries with the new ones
population_df <- population_df %>%
    filter(!(Region %in% auckland_dhbs)) %>%
    add_row(Region = 'Auckland', Link=NA, Population=auckland_population)

population_df <- population_df %>%
    filter(!(Region %in% capital_coast_hutt_dhbs)) %>%
    add_row(Region = 'Capital & Coast and Hutt Valley', Link=NA, Population=capital_hutt_population)    

population_df[population_df == "Tairāwhiti"] <- "Tairawhiti"


# Function to fix the Hawke's Bay name
fix_hawkes <- function(Region) {
    result <- ifelse(grepl('Hawk', Region, fixed=TRUE), 'Hawke\'s Bay', Region)
    return(result)
}

# Fixing the Hawkes bay names
population_df <- population_df %>% 
    mutate(Region = map(Region, fix_hawkes)) %>%
    mutate(Region = as.character(Region))

In [8]:
# Taking only the Region and Population data as the Link is not needed
population_data <- population_df %>% 
    select(Region, Population)

# Getting the border crossing data setup

The border crossing data comes from the statsnz covid19 data portal. The file has been re-uploaded to our github for ease of use.

In [9]:
border_crossing <- "https://raw.githubusercontent.com/andyholmes1210/DATA201-Project/main/covid_19_data_portal_Border%20crossings.csv" %>%
    read_csv()

Rows: 10555 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (7): class, category, indicator_name, series_name, sub_series_name, uni...
dbl  (1): value
date (1): parameter

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Getting the jobseeking data setup

The jobseeking data also comes from the StatsNZ covid19 data portal. This also uses a file that has been re-uploaded to our github for ease of use.
This section wrangles the full dataset into just the data relevant to jobseeking data

In [10]:
# Dowloading covid data into a dataframe
covid_data <- 'https://raw.githubusercontent.com/andyholmes1210/DATA201-Project/main/covid_19_data_portal.csv' %>%
   read_csv()

Rows: 297375 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): class, category, indicator_name, series_name, sub_series_name, para...
dbl (1): value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [11]:
#selecting pertinent columns from data set

`%!in%` <- Negate(`%in%`)

monthly_jobseeking_data <- covid_data %>% 
    select(class, category, indicator_name, 
           Region=series_name, Date=parameter, Jobseeker_benefit= value) %>% #renaming selected columns
    filter(class == "Income support", category == 'Benefits', indicator_name == 'Jobseeker support by MSD region', Region %!in% c('Total'))

In [12]:
#Renaming of regions to coincide with the regional population data

monthly_jobseeking_data[monthly_jobseeking_data == 'Auckland metro'] <- 'Auckland'
monthly_jobseeking_data[monthly_jobseeking_data == 'Central'] <- 'MidCentral'
monthly_jobseeking_data[monthly_jobseeking_data == 'Nelson'] <- 'Nelson Marlborough'
monthly_jobseeking_data[monthly_jobseeking_data == 'Wellington'] <- 'Capital & Coast and Hutt Valley'
monthly_jobseeking_data <- monthly_jobseeking_data %>%
    filter(Region != 'Other region') #selecting for all regions that aren't "Other region"

# Getting the health trust data setup


Combining quarterly datasets from statsnz to get the average trust in the NZ health system by Regions.

In [13]:
# function to wrangle average health trust data by region for different xlsx files so can be plotted against each other
regions = c('Northland', "Auckland", "Waikato", "Bay of Plenty", "Gisborne/Hawke's Bay", 'Taranaki', 'Manawatu-Whanganui', 'Wellington', 
            'Nelson/Tasman/Marlborough/West Coast', 'Nelson/Tasman/ Marlborough/West Coast', 'Canterbury', 'Otago', 'Southland')

extract_trust <- function(wellbeing_period){
    
    #read in wellbeing xlsx, 5th sheet(Regions), skipping first 5 rows due to NA values
    wellbeing_df <- read_xlsx(wellbeing_period, sheet=5, skip=5) 
    
    #selecting columns for regions and their data only, checking that column names are in provided region set
    regionalwb_df <- wellbeing_df %>% select_if(colnames(wellbeing_df) %in% regions) 
    
    #finding row index num for row containing data on average trust (mean row is 5 rows 'lower' than the title row)
    health_trust_row_num <- which(wellbeing_df %>% select(1) == 'Trust held for health system') + 5
    
    #dataframe of just ave rating row, using found row number
    ave_trust_df <- regionalwb_df[health_trust_row_num,] 
    
    #transforming into tidy data with Region col title and regions values in col
    ave_trust_df <- ave_trust_df %>% gather('Regions', Average_rating, 1:12) 
    
    #changing col data type from chr to dbl for plotting functionality
    ave_trust_df <- ave_trust_df %>% mutate(Average_rating = as.double(Average_rating)) 
    
    return(ave_trust_df)
}

In [14]:
download.file(url='https://raw.githubusercontent.com/andyholmes1210/DATA201-Project/main/wellbeing-statistics-March-2021-quarter.xlsx',
                destfile='March2021.xlsx', mode = 'wb')

download.file(url='https://raw.githubusercontent.com/andyholmes1210/DATA201-Project/main/wellbeing-statistics-december-2020-quarter.xlsx',
                destfile='December2020.xlsx', mode = 'wb')

download.file(url='https://raw.githubusercontent.com/andyholmes1210/DATA201-Project/main/wellbeing-statistics-september-2020-quarter.xlsx',
                destfile='September2020.xlsx', mode = 'wb')

download.file(url='https://raw.githubusercontent.com/andyholmes1210/DATA201-Project/main/wellbeing-statistics-june-2020-quarter.xlsx',
                destfile='June2020.xlsx', mode = 'wb')

In [15]:
suppressMessages(march21_trust <- extract_trust('March2021.xlsx'))
suppressMessages(dec20_trust <- extract_trust('December2020.xlsx'))
suppressMessages(sept20_trust <- extract_trust('September2020.xlsx'))
suppressMessages(june20_trust <- extract_trust('June2020.xlsx')) #hiding output of default col names

In [16]:
#fixing rouge row value in Region in Sept 2020 df
sept20_trust[sept20_trust =='Nelson/Tasman/ Marlborough/West Coast'] <- 'Nelson/Tasman/Marlborough/West Coast'


#function to add col of dates for each quarter tibble
add_date_col <- function(trust_quarter, n, value){
    date <- rep(value, n)
    trust_quarter <- trust_quarter %>% 
        add_column(date)
    return(trust_quarter)
}

In [ ]:
#giving each df a date col
march21_trust <- march21_trust %>% add_date_col(n=nrow(march21_trust), value='March 2021')
dec20_trust <- dec20_trust %>% add_date_col(n=nrow(dec20_trust), value='Dec 2020')
sept20_trust <- sept20_trust %>% add_date_col(n=nrow(sept20_trust), value='Sept 2020')
june20_trust <- june20_trust %>% add_date_col(n=nrow(june20_trust), value='June 2020')

#binding all the df's into one
trust_df <- bind_rows(june20_trust, sept20_trust, dec20_trust, march21_trust)

In [26]:
write.csv(trust_df, 'trust_data.csv', 
             row.names = FALSE)

There are now  6 different dataframes setup with all of the data that will be used for this project.

# Wrangling the data into forms useable for joins and plotting

This sections focuses on getting the above 6 dataframes wrangled so that they can be joined and the resulting data exported for plotting.

# Wrangling the vaccination data to align with the population data

In [18]:
# This cell mutates the vaccination data into a form that joins with the population data nicely and correctly (I believe)

vaccination_data <- vaccination_data_raw %>%
    select("Week ending", "Group", 'DHB of service', 'Dose number', '# doses administered') # Taking only the desired info

new_col_names = c('Week', 'Group', 'Region', 'Dose', 'Volume') # Renaming the columns for ease of use and clarity
colnames(vaccination_data) <- new_col_names

vaccination_data <- vaccination_data %>% 
    filter(Dose == 1) %>%  # Taking data on only the first dose numbers
    mutate(Region = ifelse(Region == 'Auckland Metro', 'Auckland', Region)) %>% # Renaming 'Auckland Metro' to 'Auckland'
    filter(Region != 'Other Sites') # Removing the 'Other' data 

# Function to fix the Hawke's Bay name
fix_Hawkes <- function(Region) {
    result <- ifelse(grepl('Hawk', Region, fixed=TRUE), 'Hawke\'s Bay', Region)
    return(result)
}
# Fixing the hawkes bay names
vaccination_data <- vaccination_data %>% 
    mutate(Region = map(Region, fix_Hawkes)) %>%
    mutate(Region = as.character(Region))

In [19]:
# This cell simply joins the wrangled vaccination data with the population data 
vaccination_population_data <- vaccination_data %>%
    left_join(population_data, by = 'Region')

In [20]:
# Outputting the resulting dataframe to a csv, for use in later plotting
write.csv(vaccination_population_data, file = "vaccination_population_data.csv",
          row.names = FALSE)

# Wrangling the vaccination and case data so that they align on dates (weekly)

In [21]:
# This cell sets up the vaccination data to be one observation per week per region
vaccination_weekly <- vaccination_data %>%
    select(Week, Region, Volume) %>%
    group_by(Week, Region) %>%
    summarise(Volume = sum(Volume))

`summarise()` has grouped output by 'Week'. You can override using the `.groups` argument.


In [22]:
# This is the case data modified to be weekly cases instead of daily cases
modified_case_data <- case_data %>% 
    filter(Date_reported >= vaccination_data$Week[1], Date_reported <= vaccination_data$Week[length(vaccination_data$Week)]) %>% # Only for weeks that line up with vaccinations
    mutate(week = cut.Date(Date_reported, breaks = "1 week", labels = FALSE)) %>%  # Numbering by week
    arrange(Date_reported) %>%
    group_by(week) %>% # Grouping by week number
    summarise(WeeklyCases = sum(New_cases), Week = max(Date_reported)) %>%  # Summarising weekly cases
    select(Week, WeeklyCases) # Taking only the nessacery values

# Joining the case data with the vaccination data to get weekly case and vaccine numbers
vaccination_case_data <- modified_case_data %>%
    inner_join(vaccination_weekly, by = 'Week') %>% 
    group_by(Week) %>%
    summarise(WeeklyCases = sum(WeeklyCases), Volume = sum(Volume)) %>%
    mutate(Volume = Volume/1000)

In [23]:
# Outputting the resulting dataframe to a csv, for use in later plotting
write.csv(vaccination_case_data, file = "vaccination_case_data.csv",
          row.names = FALSE)

# Wrangling the Jobseeker and population data into one dataframe

In [24]:
# merging the two sets on the regions
jobseeker_population_df <- monthly_jobseeking_data %>% 
    filter(Date != '2022-09-03') %>% 
    inner_join(population_data, by='Region')

# creating new column of the per job seeker benefit rate per capita
jobseeker_population_df<- jobseeker_population_df %>% mutate(proportion = Jobseeker_benefit/Population)
jobseeker_population_df <- jobseeker_population_df %>% mutate(Date = as.Date(Date))

# Selecting only relevant columns for plotting
jobseeker_population_df <- jobseeker_population_df %>%
    select(Date, Region, Jobseeker_benefit, Population, proportion)

In [62]:
write.csv(jobseeker_population_df, file = 'jobseeker_population_df.csv',
              row.names = FALSE)

# Wrangling the Border Crossing Data

This section will focus on wrangling the Border crossing dataset for joining with the covid case data

In [52]:
#filtering dates for covid cases
#to be able to use inner_join with border_crossing later
covid_case_new_zealand <- case_data %>%
    filter(Date_reported < "2021-10-12") %>%
    rename(Dates = 'Date_reported')

In [54]:
#removing unwanted columns
border_region <- select(border_crossing, -category, -indicator_name, -class, -units, -date_last_updated, -sub_series_name)

#rename column titles
border_region <- border_region %>% rename(
    City = series_name,
    Dates = parameter,
    Border_crossed = value)

#Getting total border crossing for each day 2020-2021
border_region_new <- border_region %>%
    filter(Dates > "2020-01-02") %>%
    group_by(Dates) %>%
    summarise(Total_border_crossed = sum(Border_crossed))

In [56]:
#Joining bordercrossing region with daily covid cases
bordercrossing_covidcase <- border_region_new %>%
    inner_join(covid_case_new_zealand, by = "Dates")

#changing any NA values to 0
bordercrossing_covidcase[is.na(bordercrossing_covidcase)] <- 0


In [67]:
write_csv(bordercrossing_covidcase, file = 'bordercrossing_covid.csv')